This is a demo of how to use 'rok_estimator.py'.

In [2]:
from rok_estimator import *
from lattice_lib import *
from lattice_lib.util import *

In [ ]:
# Specify parameters related to the underlying ring
ring_params_60_32 = RingParam(f=60,nsis=2)

# Create a new starting relation
rel = Relation(ring_params=ring_params_60_32,wdim=60)
rel.show()

Relation:
    H * F * W = Y
Statement:
    H: nout x (ntop + nbot)
    F: (ntop + nbot) x wdim
    Y: nout x rep
Witness:
    W: wdim x rep
    ||sigma(W)||_2 <= 2^log_beta_wit_2
    ||psi(W)||_inf <= 2^log_beta_wit_inf
Parameters:
    wdim = 60, rep = 1, log_beta_wit_2 = 0, log_beta_wit_inf = 0
 


/home/russell/Nextcloud/Academic/Papers/rok-paper-sissors-estimator/rok_estimator.py:107: UserWarning: Specified module rank for SIS is too small for the target security level.
  warnings.warn("Specified module rank for SIS is too small for the target security level.")


In [4]:
# Running pi_bdecomp reducing rel to rel_bdecomp
rel_bdecomp, cost_bdecomp = rel.pi_bdecomp(ell=2)
rel_bdecomp.show()
cost_bdecomp.show()

Relation:
    H * F * W = Y
Statement:
    H: nout x (ntop + nbot)
    F: (ntop + nbot) x wdim
    Y: nout x rep
Witness:
    W: wdim x rep
    ||sigma(W)||_2 <= 2^log_beta_wit_2
    ||psi(W)||_inf <= 2^log_beta_wit_inf
Parameters:
    wdim = 60, rep = 2, log_beta_wit_2 = 8, log_beta_wit_inf = 0
 
Parameter Changes:
Costs:
    communication: 1024
    soundness error: 2^-inf
 


In [5]:
# Running pi_split reducing rel into rel_split
rel_split, cost_split = rel.pi_split(2)
rel_split.show()
cost_split.show()

Relation:
    H * F * W = Y
Statement:
    H: nout x (ntop + nbot)
    F: (ntop + nbot) x wdim
    Y: nout x rep
Witness:
    W: wdim x rep
    ||sigma(W)||_2 <= 2^log_beta_wit_2
    ||psi(W)||_inf <= 2^log_beta_wit_inf
Parameters:
    wdim = 30, rep = 2, log_beta_wit_2 = 0, log_beta_wit_inf = 0
 
Parameter Changes:
Costs:
    communication: 1024
    soundness error: 2^-128
 


In [6]:
# Running pi_batch reducing rel into rel_batch
rel_batch, cost_batch = rel.pi_batch()
rel_batch.show()
cost_batch.show()

Relation:
    H * F * W = Y
Statement:
    H: nout x (ntop + nbot)
    F: (ntop + nbot) x wdim
    Y: nout x rep
Witness:
    W: wdim x rep
    ||sigma(W)||_2 <= 2^log_beta_wit_2
    ||psi(W)||_inf <= 2^log_beta_wit_inf
Parameters:
    wdim = 60, rep = 1, log_beta_wit_2 = 0, log_beta_wit_inf = 0
 
Parameter Changes:
Costs:
    communication: 0
    soundness error: 2^-128
 


In [11]:
# Running pi_fold reducing rel into rel_fold
rel_fold, cost_fold = rel.pi_fold(23)
rel_fold.show()
cost_fold.show()

Relation:
    H * F * W = Y
Statement:
    H: nout x (ntop + nbot)
    F: (ntop + nbot) x wdim
    Y: nout x rep
Witness:
    W: wdim x rep
    ||sigma(W)||_2 <= 2^log_beta_wit_2
    ||psi(W)||_inf <= 2^log_beta_wit_inf
Parameters:
    wdim = 60, rep = 23, log_beta_wit_2 = 3, log_beta_wit_inf = 8
 
Parameter Changes:
Costs:
    communication: 0
    soundness error: 2^-83
 


In [8]:
# Running pi_norm reducing rel into rel_norm
rel_norm, cost_norm = rel.pi_norm()
rel_norm.show()
cost_norm.show()


Relation:
    H * F * W = Y
Statement:
    H: nout x (ntop + nbot)
    F: (ntop + nbot) x wdim
    Y: nout x rep
Witness:
    W: wdim x rep
    ||sigma(W)||_2 <= 2^log_beta_wit_2
    ||psi(W)||_inf <= 2^log_beta_wit_inf
Parameters:
    wdim = 60, rep = 8, log_beta_wit_2 = 7, log_beta_wit_inf = 0
 
Parameter Changes:
Costs:
    communication: 46080
    soundness error: 2^-122
 
